In [1]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path, "images/*")))
    masks = sorted(glob(os.path.join(path, "masks/*")))

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [3]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([256, 256, 3])
    y.set_shape([256, 256, 1])
    return x, y

def tf_dataset(x, y, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

# model

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [5]:
def batchnorm_relu(inputs):
    """ Batch Normalization & ReLU """
    x = BatchNormalization()(inputs)
    x = Activation("relu")(x)
    return x

def residual_block(inputs, num_filters, strides=1):
    """ Convolutional Layers """
    x = batchnorm_relu(inputs)
    x = Conv2D(num_filters, 3, padding="same", strides=strides)(x)
    x = batchnorm_relu(x)
    x = Conv2D(num_filters, 3, padding="same", strides=1)(x)

    """ Shortcut Connection (Identity Mapping) """
    s = Conv2D(num_filters, 1, padding="same", strides=strides)(inputs)

    """ Addition """
    x = x + s
    return x

def decoder_block(inputs, skip_features, num_filters):
    """ Decoder Block """

    x = UpSampling2D((2, 2))(inputs)
    x = Concatenate()([x, skip_features])
    x = residual_block(x, num_filters, strides=1)
    return x

In [6]:
def build_resunet(input_shape):
    """ RESUNET Architecture """

    inputs = Input(input_shape)

    """ Endoder 1 """
    x = Conv2D(64, 3, padding="same", strides=1)(inputs)
    x = batchnorm_relu(x)
    x = Conv2D(64, 3, padding="same", strides=1)(x)
    s = Conv2D(64, 1, padding="same")(inputs)
    s1 = x + s

    """ Encoder 2, 3 """
    s2 = residual_block(s1, 128, strides=2)
    s3 = residual_block(s2, 256, strides=2)

    """ Bridge """
    b = residual_block(s3, 512, strides=2)

    """ Decoder 1, 2, 3 """
    x = decoder_block(b, s3, 256)
    x = decoder_block(x, s2, 128)
    x = decoder_block(x, s1, 64)

    """ Classifier """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(x)

    """ Model """
    model = Model(inputs, outputs, name="RESUNET")

    return model

In [7]:
shape = (256, 256, 3)
model = build_resunet(shape)
model.summary()

Model: "RESUNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 256, 256, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 256, 256, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 256, 256, 64)         0         ['batch_normalization[0]

# train

In [8]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
# from data import load_data, tf_dataset
# from model import build_model

In [9]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


In [10]:
path = "/kaggle/input/kvasirseg/Kvasir-SEG/"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)

print(len(train_x),len(train_y),len(valid_x),len(valid_y),len(test_x),len(test_y))



800 800 100 100 100 100


In [11]:
## Hyperparameters
batch = 8
lr = 1e-4
epochs = 400

train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)

for x,y in train_dataset:
    print(x.shape,y.shape)
    break;

(8, 256, 256, 3) (8, 256, 256, 1)


In [12]:
opt = tf.keras.optimizers.Adam(lr)
metrics = ["acc", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)

callbacks = [
        ModelCheckpoint("files/model.h5",monitor="val_acc", mode="max", 
                         save_best_only=True, verbose=1),
        ReduceLROnPlateau(monitor='val_loss', factor = 0.5, patience = 15, 
                         min_delta = 0.001,min_lr = 1e-10,verbose = 1),
        CSVLogger("files/data.csv"),
        TensorBoard()
    ]



In [13]:
train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch

if len(train_x) % batch != 0:
    train_steps += 1
if len(valid_x) % batch != 0:
    valid_steps += 1

model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=epochs,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks)

Epoch 1/400
100/100 [==============================] - ETA: 0s - loss: 0.4388 - acc: 0.8214 - recall: 0.1771 - precision: 0.3694 - iou: 0.1631
Epoch 1: val_acc improved from -inf to 0.83064, saving model to files/model.h5


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 90s 577ms/step - loss: 0.4388 - acc: 0.8214 - recall: 0.1771 - precision: 0.3694 - iou: 0.1631 - val_loss: 0.4124 - val_acc: 0.8306 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_iou: 0.1004 - lr: 1.0000e-04
Epoch 2/400
100/100 [==============================] - ETA: 0s - loss: 0.3359 - acc: 0.8436 - recall: 0.2180 - precision: 0.5193 - iou: 0.1966
Epoch 2: val_acc did not improve from 0.83064
100/100 [==============================] - 56s 561ms/step - loss: 0.3359 - acc: 0.8436 - recall: 0.2180 - precision: 0.5193 - iou: 0.1966 - val_loss: 0.4602 - val_acc: 0.8306 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_iou: 0.0710 - lr: 1.0000e-04
Epoch 3/400
100/100 [==============================] - ETA: 0s - loss: 0.3196 - acc: 0.8509 - recall: 0.2594 - precision: 0.5697 - iou: 0.2133
Epoch 3: val_acc did not improve from 0.83064
100/100 [==============================] - 57s 575ms/step - loss: 0.3196 - acc: 0.8509 - recall:

In [14]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm
# from data import load_data, tf_dataset
# from train import iou


In [15]:
def test_read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    return x

def test_read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = np.expand_dims(x, axis=-1)
    return x

def test_mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask


In [16]:
test_dataset = tf_dataset(test_x, test_y, batch=batch)
for x,y in test_dataset:
    print(x.shape,y.shape)
    break;

(8, 256, 256, 3) (8, 256, 256, 1)


In [17]:
test_steps = (len(test_x)//batch)
if len(test_x) % batch != 0:
    test_steps += 1

# with CustomObjectScope({'iou': iou}):
#     model = tf.keras.models.load_model("files/model.h5")

model.evaluate(test_dataset, steps=test_steps)

for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
    x = test_read_image(x)
    y = test_read_mask(y)
    y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
    h, w, _ = x.shape
    white_line = np.ones((h, 10, 3)) * 255.0

    all_images = [
            x * 255.0, white_line,
            test_mask_parse(y), white_line,
            test_mask_parse(y_pred) * 255.0
        ]
    image = np.concatenate(all_images, axis=1)
    cv2.imwrite(f"results/{i}.png", image)

13/13 [==============================] - 3s 197ms/step - loss: 1.4133 - acc: 0.8751 - recall: 0.4671 - precision: 0.7197 - iou: 0.4004


  0%|          | 0/100 [00:00<?, ?it/s]

1/1 [==============================] - 1s 1s/step


  1%|          | 1/100 [00:01<02:34,  1.56s/it]

1/1 [==============================] - 0s 20ms/step


...

 97%|█████████▋| 97/100 [00:09<00:00, 10.57it/s]

1/1 [==============================] - 0s 19ms/step


 99%|█████████▉| 99/100 [00:10<00:00, 10.62it/s]

1/1 [==============================] - 0s 20ms/step


100%|██████████| 100/100 [00:10<00:00,  9.76it/s]
